## Publication: Continous Tomography Implementation
 This markdown is the implementation of the tomobase library for usage of comparing 3 sets of data 
 1. Binomial Decomposition
 2. GRS 
 3. Incremental
 

### Imports
 This section is used to import useful code for storing data

In [1]:
from tomobase.log import logger
import numpy as np
import napari
from tomobase import data, phantoms, tiltschemes, processes
from tomobase.data import Sinogram, Volume
import pandas as pd
import os 


def get_directory():
    path = os.getcwd()
    path = os.path.join(path, '..\..\data')
    return path

def get_filename(*args):
    path = get_directory()
    path = os.path.join(get_directory(), *args)
    return path

numbers = [np.linspace(1, 65, 65),
           np.linspace(1, 33, 33),
           np.linspace(1, 129, 129)]

print(get_directory())           
vol = phantoms.nanocage()

2024-10-17 18:24:13,558 - DEBUG - This is a debug message
2024-10-17 18:24:29,753 - INFO - ProcessItemDict Initialized
INFO:tomobase_logger:ProcessItemDict Initialized
2024-10-17 18:24:29,753 - INFO - Spec Origin: d:\code\python\gitprojects\timedependenttomography\submodules\tomobase\tomobase\__init__.py
INFO:tomobase_logger:Spec Origin: d:\code\python\gitprojects\timedependenttomography\submodules\tomobase\tomobase\__init__.py
2024-10-17 18:24:29,753 - INFO - Module Path: processes\forward_project.py
INFO:tomobase_logger:Module Path: processes\forward_project.py
2024-10-17 18:24:29,753 - INFO - Module Path: processes\reconstruct.py
INFO:tomobase_logger:Module Path: processes\reconstruct.py
2024-10-17 18:24:29,753 - INFO - Module Path: processes\__init__.py
INFO:tomobase_logger:Module Path: processes\__init__.py
2024-10-17 18:24:29,769 - INFO - Module Path: processes\alignments\misalignments.py
INFO:tomobase_logger:Module Path: processes\alignments\misalignments.py
2024-10-17 18:24:30,

d:\Code\Python\GitProjects\TimeDependentTomography\scripts\continoustomo\..\..\data


### Generate TiltSeries

In [ ]:
viewer = napari.Viewer()
for i in range(len(numbers)):
    indices = numbers[i]
    scheme = tiltschemes.GRS(64.0, -64.0, 0)
    angles = np.array([scheme.get_angle() for j in indices])
    ts = processes.project(vol, angles)
    ts = processes.alignments.pad_sinogram(ts, 1024, 1024, inplace=False)
    ts.to_file(get_filename('tiltseries', 'grs_%d.mrc' % len(indices)))
    value = ts._to_napari_layer(True)
    viewer._add_layer_from_data(*value)
    
    scheme = tiltschemes.Binary(64.0, -64.0, 8)
    angles = np.array([scheme.get_angle() for j in indices])
    ts = processes.project(vol, angles)
    ts = processes.alignments.pad_sinogram(ts, 1024, 1024, inplace=False)
    ts.to_file(get_filename('tiltseries', 'bin_%d.mrc' % len(indices)))

    
    scheme = tiltschemes.Incremental(64.0, -64.0, 128/len(indices)-1)
    angles = np.array([scheme.get_angle() for j in indices])
    ts = processes.project(vol, angles)
    ts = processes.alignments.pad_sinogram(ts, 1024, 1024, inplace=False)
    ts.to_file(get_filename('tiltseries', 'inc_%d.mrc' % len(indices)))

    
    




In [31]:
# for all files in the directory
viewer = napari.Viewer()
for filename in os.listdir(get_filename('tiltseries')):
    if ('grs' in filename) and ('129' in filename):
        ts = Sinogram.from_file(get_filename('tiltseries', filename))
        #sort tilt series by ts.angle
        indices = np.argsort(ts.angles)
        ts.data = ts.data[:,:,indices]
        ts.angle = ts.angles[indices]
        print(np.max(ts.data))
        ts, shifts = processes.alignments.translational_misalignment(ts, 0.2, 0.2, extend_return=True)
        ts, shifts_corrected = processes.alignments.align_sinogram_xcorr(ts, extend_return=True)

print(shifts)
print(shifts_corrected)
offsets = (shifts_corrected + shifts)%1024
offsets[offsets>512] = np.abs(offsets[offsets>512] - 1024)
offsets.astype(int)
print(offsets)
value = ts._to_napari_layer(True)
viewer._add_layer_from_data(*value)

    

{'dimensions': array([1023, 1023,  129]), 'MRCtype': 2, 'compressor': None, 'dtype': '<f4', 'pixelsize': array([1., 1., 1.]), 'pixelunits': '\\AA', 'minImage': 0.0, 'maxImage': 0.0, 'meanImage': 0.0, 'extendedBytes': 950, 'metaId': b'json', 'voltage': 0.0, 'C3': 0.0, 'gain': 1.0, 'packedBytes': (0,), 'angles': [-64.0, 15.11, -33.78, 45.33, -3.57, -52.46, 26.65, -22.24, 56.87, 7.98, -40.92, 38.19, -10.7, -59.59, 19.52, -29.37, 49.73, 0.84, -48.05, 31.06, -17.83, 61.28, 12.38, -36.51, 42.6, -6.29, -55.18, 23.93, -24.97, 54.14, 5.25, -43.64, 35.47, -13.42, -62.32, 16.79, -32.1, 47.01, -1.88, -50.77, 28.33, -20.56, 58.55, 9.66, -39.23, 39.88, -9.02, -57.91, 21.2, -27.69, 51.42, 2.53, -46.37, 32.74, -16.15, 62.96, 14.07, -34.82, 44.28, -4.61, -53.5, 25.61, -23.28, 55.83, 6.93, -41.96, 37.15, -11.74, -60.63, 18.48, -30.42, 48.69, -0.2, -49.09, 30.02, -18.87, 60.23, 11.34, -37.55, 41.56, -7.33, -56.22, 22.88, -26.01, 53.1, 4.21, -44.68, 34.43, -14.47, -63.36, 15.75, -33.14, 45.97, -2.92, -51.

[<Image layer 'Sinogram' at 0x2e09954f010>]

In [30]:
np.set_printoptions(suppress=True, precision=3)
print(offsets)
value = np.sum(offsets, axis=0)
value = value/len(offsets)
print(value)

[[0. 0.]
 [0. 0.]
 [0. 1.]
 [1. 1.]
 [0. 0.]
 [0. 0.]
 [1. 0.]
 [1. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 0.]
 [1. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 1.]
 [0. 1.]
 [0. 0.]
 [0. 0.]
 [1. 0.]
 [0. 0.]
 [1. 0.]
 [0. 1.]
 [1. 1.]
 [0. 0.]
 [1. 1.]
 [0. 0.]
 [1. 1.]
 [0. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [0. 0.]
 [0. 1.]
 [0. 0.]
 [1. 1.]
 [0. 1.]
 [1. 0.]
 [0. 0.]
 [1. 0.]
 [1. 1.]
 [1. 0.]
 [0. 0.]
 [1. 1.]
 [1. 1.]
 [1. 0.]
 [1. 0.]
 [0. 0.]
 [1. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 1.]
 [0. 1.]
 [1. 0.]
 [1. 1.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 1.]
 [0. 0.]
 [1. 1.]
 [1. 0.]
 [1. 1.]
 [0. 0.]
 [1. 0.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [0. 0.]
 [1. 0.]
 [0. 1.]
 [1. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 1.]
 [1. 0.]
 [1. 1.]
 [0. 1.]
 [1. 0.]
 [0. 0.]
 [0. 1.]
 [1. 0.]
 [1. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 0.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 0.]
 [1. 0.]
 [1. 1.]
 [0. 1.]
 [0. 0.]
 [1. 0.]
 [0. 0.]
 [1. 1.]
 [1. 1.]
 

### Get Tilt Series 


In [ ]:
# for each tilt series pad till 1024 by 1024
import numpy as np

print("Original shapes:")
print("ts_grs.data.shape:", ts_grs.data.shape)
print("ts_bin.data.shape:", ts_bin.data.shape)

# Pad the first two dimensions to 1024 while keeping the third dimension unchanged
ts_grs.data = np.pad(ts_grs.data, ((0, 1024 - ts_grs.data.shape[0]), (0, 1024 - ts_grs.data.shape[1]), (0, 0)), 'constant')
ts_grs.data = np.pad(ts_bin.data, ((0, 1024 - ts_bin.data.shape[0]), (0, 1024 - ts_bin.data.shape[1]), (0, 0)), 'constant')


ts_grs_misaligned, offset_grs =  processes.alignments.translational_misalignment(ts_grs, 0.1, inplace=False, extend_return=True)
ts_binary_misaligned, offset_binary = processes.alignments.translational_misalignment(ts_bin, 0.1, inplace =False, extend_return=True)

In [4]:

ts_grs_aligned = processes.alignments.align_sinogram_xcorr(ts_grs_misaligned, inplace=False, extend_return=True)
ts_bin_aligned = processes.alignments.align_sinogram_xcorr(ts_binary_misaligned, inplace=False, extend_return = True)

In [ ]:
print(offset_binary)
print(ts_bin_aligned[1])

In [ ]:
import numpy as np
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from skimage.filters import threshold_otsu

name  = 'compare-nomask'
volume = volume_increment.data
vol = volume_grs

#threshold_value = threshold_otsu(volume.flatten())
#volume_masked = np.where(volume.data > threshold_value, volume, 0)
volume_masked = volume

psnr_value = psnr(vol.data, volume_masked, data_range =1.0)
ssim_value, _ = ssim(vol.data, volume_masked, full=True, multichannel=True, data_range =1.0)
if os.path.exists(os.path.join(save_path, 'psnr_ssims.csv')):
    df = pd.read_csv(os.path.join(save_path,'psnr_ssims.csv'))
else:
    df = pd.DataFrame()
data = {
    'Name': name,
    'PSNR': psnr_value,
    'SSIM': ssim_value
}
print('PSNR: {}'.format(psnr_value))
print('SSIM: {}'.format(ssim_value))

df2 = pd.DataFrame(data, index=[0])
df = pd.concat([df, df2])
df.to_csv(os.path.join(save_path, 'psnr_ssims.csv'), index=False)

